In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, StratifiedKFold
from imblearn.pipeline import Pipeline as ImPipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import RFECV
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1, L2
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime
from xgboost import XGBRegressor
%matplotlib inline

In [4]:
merge_df = pd.read_csv('../data/merged_fftr.csv')
month_df = pd.read_csv('../data/monthly_data.csv')

In [5]:
merge_df = merge_df.set_index('DATE')
month_df = month_df.set_index('DATE')

In [6]:
merge_df.head()

,FFTR,UNRATE,Oil,PCE,DispInc,gdp_gap,GDP,EXPGS,IMPGS,ind_prod,...,gbp_dollar,pct_recession,yen_dollar,yield_curve,1_month_ahead,2_month_ahead,3_month_ahead,1_month_back,2_month_back,3_month_back
DATE,,,,,,,,,,,,,,,,,,,,,
1987-11-09,6.8125,5.773333,17.50,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7930,0.02,134.30,1.140,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-10,6.8125,5.770000,17.75,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7865,0.02,134.55,1.180,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-11,6.8125,5.766667,17.80,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7720,0.02,135.50,1.155,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-12,6.8125,5.763333,17.85,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7575,0.02,136.45,1.130,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-13,6.8125,5.760000,17.80,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7680,0.02,135.85,1.090,7.3125,7.25,6.625,6.8125,6.8125,6.500


In [7]:
merge_df.head()

,FFTR,UNRATE,Oil,PCE,DispInc,gdp_gap,GDP,EXPGS,IMPGS,ind_prod,...,gbp_dollar,pct_recession,yen_dollar,yield_curve,1_month_ahead,2_month_ahead,3_month_ahead,1_month_back,2_month_back,3_month_back
DATE,,,,,,,,,,,,,,,,,,,,,
1987-11-09,6.8125,5.773333,17.50,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7930,0.02,134.30,1.140,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-10,6.8125,5.770000,17.75,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7865,0.02,134.55,1.180,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-11,6.8125,5.766667,17.80,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7720,0.02,135.50,1.155,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-12,6.8125,5.763333,17.85,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7575,0.02,136.45,1.130,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-13,6.8125,5.760000,17.80,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7680,0.02,135.85,1.090,7.3125,7.25,6.625,6.8125,6.8125,6.500


In [8]:
merge_df = merge_df.iloc[90:]
merge_df = merge_df.drop(merge_df.tail(90).index)

In [9]:
month_df = month_df.iloc[3:]
month_df = month_df.drop(month_df.tail(3).index)

In [10]:
month_df.head()

,FFTR,UNRATE,Oil,PCE,DispInc,gdp_gap,GDP,EXPGS,IMPGS,ind_prod,...,gbp_dollar,pct_recession,yen_dollar,yield_curve,1_month_ahead,2_month_ahead,3_month_ahead,1_month_back,2_month_back,3_month_back
DATE,,,,,,,,,,,,,,,,,,,,,
1988-05-01,6.7500,5.6,16.166667,4.9,6841.7,-0.037442,5190.036,439.471,546.098,60.9952,...,1.873833,0.12,124.94,1.153333,6.7500,6.50,6.625,7.2500,7.5000,7.6875
1988-06-01,7.2500,5.4,16.330000,4.9,6868.5,-0.037442,5190.036,439.471,546.098,61.1697,...,1.823500,0.00,125.20,0.940000,6.7500,6.75,6.500,7.5000,7.6875,8.1250
1988-07-01,7.5000,5.4,13.950000,4.7,6898.2,-0.230370,5282.835,453.586,552.849,61.1749,...,1.701000,0.06,134.10,0.790000,7.2500,6.75,6.750,7.6875,8.1250,8.1250
1988-08-01,7.6875,5.6,15.600000,4.7,6918.4,-0.230370,5282.835,453.586,552.849,61.4851,...,1.713000,0.04,132.80,0.710000,7.5000,7.25,6.750,8.1250,8.1250,8.1250
1988-09-01,8.1250,5.4,14.150000,4.7,6932.4,-0.230370,5282.835,453.586,552.849,61.2844,...,1.669000,0.02,136.35,0.570000,7.6875,7.50,7.250,8.1250,8.1250,8.3750


In [11]:
X = merge_df.drop(columns = ['1_month_ahead', '2_month_ahead', '3_month_ahead'])
y_1 = merge_df['1_month_ahead']
y_2 = merge_df['2_month_ahead']
y_3 = merge_df['3_month_ahead']
X_month = month_df.drop(columns = ['1_month_ahead', '2_month_ahead', '3_month_ahead'])
y_month_1 = month_df['1_month_ahead']
y_month_2 = month_df['2_month_ahead']
y_month_3 = month_df['3_month_ahead']

In [12]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y_1, random_state=42, test_size=0.25)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y_2, random_state=42, test_size=0.25)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y_3, random_state=42, test_size=0.25)
X_train_month_1, X_test_month_1, y_train_month_1, y_test_month_1 = train_test_split(X_month, y_month_1, random_state=42, test_size=0.25)
X_train_month_2, X_test_month_2, y_train_month_2, y_test_month_2 = train_test_split(X_month, y_month_2, random_state=42, test_size=0.25)
X_train_month_3, X_test_month_3, y_train_month_3, y_test_month_3 = train_test_split(X_month, y_month_3, random_state=42, test_size=0.25)

In [13]:
grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [5,6,7]
}
print(datetime.now())
rfr_CV = GridSearchCV(estimator=RandomForestRegressor(criterion='squared_error'), param_grid=grid, cv= 5)
rfr_CV.fit(X_train_month_1, y_train_month_1)
print(datetime.now())

2024-02-29 14:25:00.905746


KeyboardInterrupt: 

In [ ]:
rfr_CV.best_params_

In [ ]:
best_rfr_model = rfr_CV.best_estimator_

In [ ]:
prediction = best_rfr_model.predict(X_test_month_1)
mse = mean_squared_error(y_test_month_1, prediction)
rmse = mse**.5
print(mse)
print(rmse)

In [ ]:
grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [2,3,4,5,6,7]
}
print(datetime.now())
rfr_CV = GridSearchCV(estimator=RandomForestRegressor(criterion='squared_error'), param_grid=grid, cv= 5)
rfr_CV.fit(X_train_month_2, y_train_month_2)
print(datetime.now())

In [ ]:
rfr_CV.best_params_

In [ ]:
best_rfr_model = rfr_CV.best_estimator_

In [ ]:
prediction = best_rfr_model.predict(X_test_month_2)
mse = mean_squared_error(y_test_month_2, prediction)
rmse = mse**.5
print(mse)
print(rmse)

In [ ]:
grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [2,3,4,5,6,7]
}
print(datetime.now())
rfr_CV = GridSearchCV(estimator=RandomForestRegressor(criterion='squared_error'), param_grid=grid, cv= 5)
rfr_CV.fit(X_train_month_3, y_train_month_3)
print(datetime.now())

In [ ]:
rfr_CV.best_params_

In [ ]:
best_rfr_model = rfr_CV.best_estimator_

In [ ]:
prediction = best_rfr_model.predict(X_test_month_3)
mse = mean_squared_error(y_test_month_3, prediction)
rmse = mse**.5
print(mse)
print(rmse)

In [ ]:
grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [4,5,6,7]
}
print(datetime.now())
rfr_CV = GridSearchCV(estimator=RandomForestRegressor(criterion='squared_error'), param_grid=grid, cv= 5)
rfr_CV.fit(X_train_1, y_train_1)
print(datetime.now())

In [ ]:
best_rfr_model = rfr_CV.best_estimator_

In [ ]:
prediction = best_rfr_model.predict(X_test_1)
mse = mean_squared_error(y_test_1, prediction)
rmse = mse**.5
print(mse)
print(rmse)

In [ ]:
grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [4,5,6,7]
}
print(datetime.now())
rfr_CV = GridSearchCV(estimator=RandomForestRegressor(criterion='squared_error'), param_grid=grid, cv= 5)
rfr_CV.fit(X_train_2, y_train_2)
print(datetime.now())

In [ ]:
best_rfr_model = rfr_CV.best_estimator_

In [ ]:
prediction = best_rfr_model.predict(X_test_2)
mse = mean_squared_error(y_test_2, prediction)
rmse = mse**.5
print(mse)
print(rmse)

In [ ]:
grid = { 
    'n_estimators': [1000, 1200],
    'max_features': ['sqrt','log2'],
    'max_depth' : [13, 14]
}
print(datetime.now())
rfr_CV = GridSearchCV(estimator=RandomForestRegressor(criterion='squared_error'), param_grid=grid, cv= 5)
rfr_CV.fit(X_train_3, y_train_3)
print(datetime.now())

In [ ]:
best_rfr_model = rfr_CV.best_estimator_

In [ ]:
MAE = mean_absolute_error(y_test_3, 
                          y_pred)
print(MAE)

In [ ]:
prediction = best_rfr_model.predict(X_test_3)
mse = mean_squared_error(y_test_3, prediction)
rmse = mse**.5
print(mse)
print(rmse)

In [ ]:
rfr_CV.best_params_

In [ ]:
r2_score(y_test_3, y_pred)

In [ ]:
X_train_3.head()

In [ ]:
np.sqrt(np.nanmean(np.square(((y_test_3 - prediction) / y_test_3))))*100

In [ ]:
grid = {'max_depth': [5,6,7], 'learning_rate': [.1, .2],
        'n_estimators': [600, 700]}
xgb_cv = GridSearchCV(estimator = XGBRegressor(objective='reg:squarederror'), scoring='neg_mean_absolute_error',
                  param_grid = grid, cv = 5)
xgb_cv.fit(X_train_3, y_train_3);

In [ ]:
xgb_cv.best_estimator_

In [ ]:
best_xgb_model = xgb_cv.best_estimator_
best_xgb_model.fit(X_train_3, y_train_3);
y_pred = best_xgb_model.predict(X_test_3)

In [ ]:
MAE = mean_absolute_error(y_test_3, 
                          y_pred)

In [ ]:
print(MAE)

In [ ]:
RMSE = np.sqrt(mean_squared_error(y_test_3,
                                  y_pred))
print(RMSE)

In [ ]:
r2_score(y_test_3, y_pred)

In [15]:
#making linreg model for comparison
linreg = LinearRegression()
linreg.fit(X_train_3, y_train_3)
y_pred = linreg.predict(X_test_3)
linreg.score(X_test_3,y_test_3)

0.9842549826324173

In [17]:
MAE = mean_absolute_error(y_test_3, 
                          y_pred)
MAE

0.22042513793634963

In [ ]:
#pretty good, but the improvement of over 1% for the random forest is amazing improvement, most notably the drop of MAE from 

In [ ]:
#gonna make a taylor rule baseline linreg model to compare to theory

In [18]:
merge_df.columns

Index(['FFTR', 'UNRATE', 'Oil', 'PCE', 'DispInc', 'gdp_gap', 'GDP', 'EXPGS',
       'IMPGS', 'ind_prod', 'm2', 'm2_velo', 'nasdaq', 'pci', 'gbp_dollar',
       'pct_recession', 'yen_dollar', 'yield_curve', '1_month_ahead',
       '2_month_ahead', '3_month_ahead', '1_month_back', '2_month_back',
       '3_month_back'],
      dtype='object')

In [22]:
X_train_taylor = X_train_3.drop(columns = ['UNRATE', 'Oil', 'PCE', 'DispInc', 'GDP', 'EXPGS', 'IMPGS', 'ind_prod', 'm2', 'm2_velo', 'nasdaq', 'gbp_dollar', 'pct_recession', 'yen_dollar', 'yield_curve', '1_month_back', '2_month_back', '3_month_back'])
X_train_taylor.head()

,FFTR,gdp_gap,pci
DATE,,,
1999-05-06,4.75,1.605682,2.153318
2022-04-06,0.50,-0.339047,4.703935
2011-02-15,0.25,-3.974206,1.117829
2013-11-19,0.25,-2.599131,1.890963
2007-05-28,5.25,0.892907,2.774199


In [24]:
X_test_taylor = X_test_3.drop(columns = ['UNRATE', 'Oil', 'PCE', 'DispInc', 'GDP', 'EXPGS', 'IMPGS', 'ind_prod', 'm2', 'm2_velo', 'nasdaq', 'gbp_dollar', 'pct_recession', 'yen_dollar', 'yield_curve', '1_month_back', '2_month_back', '3_month_back'])
X_test_taylor.head()

,FFTR,gdp_gap,pci
DATE,,,
2006-10-19,5.25,0.964888,3.285968
2021-04-19,0.25,-0.216935,2.209393
2021-01-24,0.25,-1.242469,1.400569
2011-03-19,0.25,-3.974206,1.179972
2003-07-24,1.00,-1.287599,2.228217


In [25]:
linreg = LinearRegression()
linreg.fit(X_train_taylor, y_train_3)
y_pred = linreg.predict(X_test_taylor)
linreg.score(X_test_taylor,y_test_3)

0.969202428567518

In [26]:
MAE = mean_absolute_error(y_test_3, 
                          y_pred)
MAE

0.31096647401476124

In [27]:
r2_score(y_test_3, y_pred)

0.969202428567518

In [ ]:
#just need to make visualizations of rgeression predictions